In [8]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense

In [2]:
train = pd.read_csv('./bike_data/train.csv', parse_dates=['datetime'])
test = pd.read_csv('./bike_data/test.csv', parse_dates=['datetime'])
sub = pd.read_csv('./bike_data/sampleSubmission.csv')

train.shape, test.shape, sub.shape

((10886, 12), (6493, 9), (6493, 2))

In [3]:
print(train.info())
print()
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 1020.7 KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dt

In [4]:
input_col = ['weather', 'temp']
labeled_col = ['count']

In [5]:
X = train[input_col]
y = train[labeled_col]
X_val = test[input_col]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [7]:
# 난수 발생 패턴 결정 0
seed = 0
np.random.seed(seed)

In [9]:
model = Sequential()
model.add(Dense(30, input_dim=2, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                90        
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 15)                240       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 538us/step - loss: 36655.3047
Epoch 2/20
817/817 [==============================] - 0s 531us/step - loss: 27943.1953
Epoch 3/20
817/817 [==============================] - 0s 525us/step - loss: 27851.3086
Epoch 4/20
817/817 [==============================] - 0s 487us/step - loss: 27803.5410
Epoch 5/20
817/817 [==============================] - 0s 467us/step - loss: 27781.8496
Epoch 6/20
817/817 [==============================] - 0s 509us/step - loss: 27707.9062
Epoch 7/20
817/817 [==============================] - 0s 497us/step - loss: 27666.4883
Epoch 8/20
817/817 [==============================] - 0s 544us/step - loss: 27666.5020
Epoch 9/20
817/817 [==============================] - 0s 534us/step - loss: 27622.4512
Epoch 10/20
817/817 [==============================] - 0s 529us/step - loss: 27617.7090
Epoch 11/20
817/817 [==============================] - 0s 520us/step - loss: 27612.9727
Epoch 12/20
817/817 [====================

In [12]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 468us/step - loss: 27296.5840


27296.583984375